# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset:

Import the necessary libraries and create your dataframe(s).

In [47]:
import pandas as pd
import numpy as np
from pyzipcode import ZipCodeDatabase #I looked for libraries that work well with ZIP codes and found this. There is some documentation and examples at https://pypi.org/project/pyzipcode/
zcdb = ZipCodeDatabase()

prcl = pd.read_csv("Prcl_7th_Ward.csv") #Note I pre-filtered the original database (in Access) to only be properties in the 7th Ward, since the file was too large with all the wards.
forestry = pd.read_csv("forestry-maintenance-properties.csv") #This dataframe has information on the properties that the city considers vacant and mows their lawns, I can connect it with the previous dataframe on PARCELID

#Note I'm going to merge my to data sets into one for the rest of these exercises:

print(prcl.shape)
df = prcl.merge(forestry, how="left", left_on='AsrParcelId', right_on='PARCELID') #I'm using a Left Merge, because I only want the data for the properties that are in the Parcel DF and the rest is not needed.
print(df.shape)
#It added 4 columns and no rows, so it looks like it did it right!)
print(df.columns.to_list())
#I will drop many of my columns in a future step (below)
# df.to_csv('combined_df.csv', index=False)

(8526, 187)
(8526, 191)
['CityBlock', 'Parcel', 'OwnerCode', 'AsrParcelId', 'ColParcelId', 'ColCityBlock', 'ColParcel', 'ParcelId', 'PrimAddrRecNum', 'AddrType', 'LowAddrNum', 'LowAddrSuf', 'HighAddrNum', 'HighAddrSuf', 'NLC', 'Parity', 'StPreDir', 'StName', 'StType', 'StSufDir', 'StdUnitNum', 'OwnerName', 'OwnerName2', 'OwnerAddr', 'OwnerCity', 'OwnerState', 'OwnerCountry', 'OwnerZIP', 'OwnerRank', 'LegalDesc1', 'LegalDesc2', 'LegalDesc3', 'LegalDesc4', 'LegalDesc5', 'AsrClassCode', 'PropertyClassCode', 'AsrLandUse1', 'AsrLanduse2', 'IsAbatedProperty', 'AbatementType', 'AbatementStartYear', 'AbatementEndYear', 'RedevPhase', 'RedevYearEnd', 'RedevPhase2', 'RedevYearEnd2', 'VacantLot', 'SpecBusDist', 'SpecBusDist2', 'SpecBusDist3', 'TIFDist', 'LendingAgcy', 'Condominium', 'NbrOfUnitsSource', 'NbrOfUnits', 'NbrOfApts', 'Frontage', 'LandArea', 'RecDailyDate', 'RecDailyNum', 'RecBookNum', 'RecPageNum', 'AsdLand', 'AsdImprove', 'AsdTotal', 'AsdResLand', 'AsdResImprove', 'AsdComLand', 'AsdCo

C:\Users\LC Harrison Sand\AppData\Local\Temp\ipykernel_8968\3036625950.py:6: DtypeWarning: Columns (9,27,97,107,171) have mixed types. Specify dtype option on import or set low_memory=False.
  prcl = pd.read_csv("Prcl_7th_Ward.csv") #Note I pre-filtered the original database (in Access) to only be properties in the 7th Ward, since the file was too large with all the wards.


## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [48]:
cat_count= df['CATEGORY'].count() #Tells me how many locations are on the mowing list in the new data set.
vac_count = df.loc[df['VacantLot'] == True]['VacantLot'].count() #Tells me how many locations are listed as VacantLot == True, originally from PRCL Data.
vac_cat_count = df.loc[(df['VacantLot'] == True) & (df['CATEGORY'].notnull())]['VacantLot'].count() # Tells me how many locations have both True value for VacantLot and have data in CATEGORY
vac_nocat_count = df.loc[(df['VacantLot'] == True) & (df['CATEGORY'].isnull())]['VacantLot'].count() #Vacant Lot, but not mowing schedule
novac_cat_count = df.loc[(df['VacantLot'] == False) & (df['CATEGORY'].notnull())]['VacantLot'].count() #Not Vacant Lot, but on the Mowing Schedule
type_cat_novl = df.loc[(df['VacantLot'] == False) & (df['CATEGORY'].notnull())]['CATEGORY'].unique() #What categories are the FALSE vacant lot's in the mowing schedule
type_cat_truevl = df.loc[(df['VacantLot'] == True) & (df['CATEGORY'].notnull())]['CATEGORY'].unique() #What categories are True vacantlot in the mowing schedule

mow_ownerzips_count = len(df.loc[(df['CATEGORY'].notnull()) & (df['OwnerZIP'].notnull())])
mow_owners_without_zips = df.loc[df['CATEGORY'].notnull() & df['OwnerZIP'].isnull()]['OwnerName'].unique() #I'm looking for who own's the properties that do not have ZIP Codes associated with them.

df['TaxBalDue'] = df['TaxBalDue'].replace(',','', regex=True) #Gets rid of commas to allow me to change it to a float - found here: https://saturncloud.io/blog/how-to-handle-the-pandas-valueerror-could-not-convert-string-to-float/
df['TaxBalDue'] = df['TaxBalDue'].astype('float') #Had to convert this to numbers, since it was not typed as a numeric values
mow_tax_due_count = df.loc[(df['CATEGORY'].notnull()) & df['TaxBalDue'] > 0]['TaxBalDue'].count()
mow_tax_due_sum = df.loc[(df['CATEGORY'].notnull()) & df['TaxBalDue'] > 0]['TaxBalDue'].sum()

#Based on the above I am not going to use the category of Vacantlot's since I am not sure what it is measuring, but I understand what the Mowing Schedule is.

#Printed everything to make it easier to read and understant.
print("Vacant Lots v. Mowing Categories")
print(f"Mowing schedule -{cat_count}")
print(f"Vacant Lots- {vac_count}") #Looking to see if 'VacantLot' is equivalent to 'CATEGORY'
print(f"Vacantlots and on mowing schedule - {vac_cat_count}")
print(f"Vacantlots, but not on Mowing Schedule - {vac_nocat_count}")
print(f"Not vacantlot, but on mowing - {novac_cat_count}")
print("\n")
print(f"Categories on Mowing Schedule that are listed as Vacant Lots: \n {type_cat_truevl}")
print(f"\n Categories on Mowing Schedule that are not listed as Vacant Lots: \n {type_cat_novl}")
print("\n \n")
print("Mowing Data:")
print(f"Number of properties that are on mowing schedule that have Owner ZIP listed - {mow_ownerzips_count}")
print(f"Names of Owner's of properties with no ZIP listed - {mow_owners_without_zips}")

"""Based on above the properties without ZIPCodes are owned by "'STATE OF MISSOURI' 'STATE OF MO' 'CITY OF ST LOUIS'
'HOLY CROSS EVANGELICAL LUTHERAN' 'ST LOUIS UNIVERSTIY HOSP'" - For the state and city properties I will exclude and drop them, I will find ZIP Codes for the other ones.

I will give the following values for ZIPS:
STATE OF MISSOURI - Drop Rows
STATE OF MO -  Drop Rows
CITY OF ST LOUIS - Drop Rows
I decided to drop the above, since they are in a different category of property than my business question is looking for.

HOLY CROSS EVANGELICAL LUTHERAN - 63118 from https://www.holycrossstl.org/
ST LOUIS UNIVERSTIY HOSP -  63117 from Google Search of SSM Administrative Offices
"""

print(f"Amount of locations on Mowing Schedule that owe property taxes - {mow_tax_due_count}")
print(f"Amount these property taxes due- {mow_tax_due_sum}")

#Below fixes this missing data:

#Dropping Rows
to_drop_locs = ['STATE OF MISSOURI', 'STATE OF MO', 'CITY OF ST LOUIS']

for loc in to_drop_locs:
    df = df.loc[df['OwnerName'] != loc]

df.loc[df["OwnerName"] == 'HOLY CROSS EVANGELICAL LUTHERAN', 'OwnerZIP'] = 63118
df.loc[df["OwnerName"] == 'ST LOUIS UNIVERSTIY HOSP', 'OwnerZIP'] = 63116

Vacant Lots v. Mowing Categories
Mowing schedule -1050
Vacant Lots- 974
Vacantlots and on mowing schedule - 622
Vacantlots, but not on Mowing Schedule - 352
Not vacantlot, but on mowing - 428


Categories on Mowing Schedule that are listed as Vacant Lots: 
 ['Vacant Lot' 'Fenced & Locked or Paved Lot' 'Primary Highway Easement'
 'Vacant Building' 'Mow to Own' 'Garden Lot' 'Demolition' 'Garden Lot '
 'Playground']

 Categories on Mowing Schedule that are not listed as Vacant Lots: 
 ['Fenced & Locked or Paved Lot' 'Vacant Building' 'Vacant Lot'
 'Mow to Own' 'Commercial' 'Primary Highway Easement' 'Rehab' 'Demolition'
 'Garden Lot']

 

Mowing Data:
Number of properties that are on mowing schedule that have Owner ZIP listed - 1014
Names of Owner's of properties with no ZIP listed - ['STATE OF MISSOURI' 'STATE OF MO' 'CITY OF ST LOUIS'
 'HOLY CROSS EVANGELICAL LUTHERAN' 'ST LOUIS UNIVERSTIY HOSP']
Amount of locations on Mowing Schedule that owe property taxes - 151
Amount these property 

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [49]:
'''
From EDA steps, It looks like non-vacant building have been listed with a 0 in VacBldgYear, I will need to set these as nulls. I also need to look to make sure Zip Codes are appropriate too.
'''
print(df.loc[df["VacBldgYear"]== 0]["VacBldgYear"].count())
df.loc[df["VacBldgYear"] == 0, 'VacBldgYear'] = np.nan
print(df.loc[df["VacBldgYear"]== 0]["VacBldgYear"].count())


6291
0


## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [50]:
'''
My df data has many columns that I will not use, it is easier to make a new DataFrame with the columns I want rather than drop the columns I don't want.
I did this in the previous EDA step, but will do it again with less columns.
'''
df = df.loc[:, ['CATEGORY', 'OwnerName', 'StPreDir', 'StName', 'StType', 'StSufDir', 'StdUnitNum', 'OwnerName', 'OwnerName2', 'OwnerAddr', 'OwnerCity', 'OwnerState', 'OwnerCountry', 'OwnerZIP', 'OwnerRank', 'LandArea', 'Zoning', 'VacBldgYear', 'Ward20', 'Precinct20', 'TaxBalDue']]
#I probably kept more than I need, but I might want to see some of this data at the end.


## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [52]:
'''Zip Codes have inconsistent data - some are 5 digits and some are 5+4. I will delete the +4 from any rows that have them.'''
print(df['OwnerZIP'].unique())

print(print(df['OwnerZIP'].dtypes)) #Checking what type 'OwnerZip' is (It was Object)
df['OwnerZIP'] = pd.to_numeric(df['OwnerZIP'], downcast="integer" ,errors='coerce') #I found to_numeric allows me to make things such as strings into null values, which I want. That is what errors='coerce' does. The downcast forces it into integers, which is what I want.

#The below: If the ZIP is 4 long it adds a leading 0, since leading 0s look to have been removed at some point and some ZIPs start with 0, 
# if it is longer than 5, it has the 5+4 of ZIPs and I only want the 5, so this truncates it.

for i in df.index:
    if len(df.loc[i, ['OwnerZIP']]) == 4:
        zip_adder = int(str(0 + df.loc[i,'OwnerZIP']))
        df.loc[i, 'OwnerZIP'] = zip_adder
    elif len(df.loc[i, ['OwnerZIP']]) > 5:
        zip_adder = int(str(df.loc[i,'OwnerZIP'])[0:4])
        df.loc[i, 'OwnerZIP'] = zip_adder

print(df['OwnerZIP'].unique())

[6.31120000e+04 6.30260000e+04 6.31040000e+04 6.31180000e+04
 6.30700000e+04 5.54080000e+04 6.31210000e+04 3.39900000e+04
 6.31030000e+04 6.31100000e+04 6.31070000e+04 6.20400000e+04
 6.31360000e+04 6.31390000e+04 6.31050000e+04 6.31110000e+04
 6.30250000e+04 8.34400000e+04 6.30060000e+04 6.30330000e+04
 6.31630000e+04 6.31270000e+04 1.87600000e+03 6.31410000e+04
 6.31280000e+04 6.31580000e+04 6.33030000e+04 6.31250000e+04
 6.31010000e+04 6.31310000e+04 6.31460000e+04 6.31160000e+04
 6.30140000e+04 6.31200000e+04 9.45600000e+04 9.45380000e+04
 6.31260000e+04 6.31090000e+04 6.31150000e+04 6.31380000e+04
 6.06010000e+04 6.31080000e+04 4.50690000e+04 6.33040000e+04
 6.31430000e+04 7.01190000e+04 6.31350000e+04 6.30340000e+04
 9.17650000e+04 6.30170000e+04 6.31220000e+04 6.31290000e+04
            nan 6.31440000e+04 6.31300000e+04 6.31320000e+04
 6.29580000e+04 3.10150000e+04 6.31230000e+04 6.31240000e+04
 7.01180000e+04 6.41080000e+04 6.33760000e+04 6.31060000e+04
 9.08020000e+04 6.317700

## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset?

Yes, I did. I had some missing ZIP codes, I had an irregular amount of 0s for 'VacBldgYear', which I believe just represented null data. I had lots of unecessary columns and I had ZIP codes that were either 5+4 or missing leading zeroes. 

2. Did the process of cleaning your data give you new insights into your dataset?

Yes, it helped me to decide that 'VacantLot' was not helpful for me and to instead use 'CATEGORY'. I'm still not sure what 'VacantLot' represents, but it does not help me with my issue. Cleaning the data allowed me to see this. I went back and changed checkpoint-two because of this.

3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?

The only thing to note is that I kept more columns than I need, but did not as thoroughly clean those columns. If needed I will have to go back and clean those columns.